In [1]:
import xarray as xr
import numpy as np
import geocat.comp as geocomp

In [14]:
cdnc = xr.open_dataset(snakemake.input.cdnc[0])
ta = xr.open_dataset(snakemake.input.ta[0])
dvar='cdncvi'
p2 = snakemake.params.get('p1', 10000)
attrs = cdnc.attrs.copy()

In [4]:
if snakemake.rule != 'column_integrate_cdnc_UKESM':
    with xr.set_options(keep_attrs=True):
        if snakemake.wildcards.model == 'CNRM-ESM2-1':
            dim = cdnc.dims
            b_bnds = cdnc['b_bnds'].values.ravel().reshape((dim['lev'], dim['bnds']))
            a_bnds = cdnc['ap_bnds'].values.ravel().reshape((dim['lev'], dim['bnds']))
            a_bnds = xr.DataArray(a_bnds, coords={'lev':cdnc.lev}, dims=['lev', 'bnds'])
            b_bnds = xr.DataArray(b_bnds, coords={'lev':cdnc.lev}, dims=['lev', 'bnds'])
            cdnc = cdnc.assign(ap_bnds=a_bnds,b_bnds=b_bnds)
        elif snakemake.wildcards.model == 'IPSL-CM6A-LR-INCA':
            dim = cdnc.dims
            cdnc = cdnc.cf.guess_coord_axis()
            if 'time' in cdnc['b_bnds'].dims:
                b_bnds = cdnc['b_bnds'].mean(dim='time').transpose()
                a_bnds = cdnc['ap_bnds'].mean(dim='time').transpose()
                b = cdnc['b'].mean(dim='time')
                ap=cdnc['ap'].mean(dim='time')
            else:
                b_bnds = cdnc['b_bnds'].transpose()
                a_bnds = cdnc['ap_bnds'].transpose()
                b = cdnc['b']
                ap=cdnc['ap']
            cdnc = cdnc.assign(ap_bnds=a_bnds,b_bnds=b_bnds, b = b, 
                                 ap=cdnc['ap'])
            cdnc = cdnc.cf.add_bounds('lev')
            cdnc['lev_bounds'] = cdnc['lev_bounds'].transpose().swap_dims(bounds='bnds')
            cdnc['lev'].attrs['formula'] = 'p=lev'
            cdnc = cdnc.cf.guess_coord_axis()
            cdnc = cdnc.rename({'lev':'plev'})
            
    if 'ap' in cdnc.data_vars:
        cdnc = cdnc.rename_vars({'ap': 'a'})

    if 'lev_bounds' in cdnc.data_vars:
        cdnc = cdnc.rename_vars({'lev_bounds': 'lev_bnds'})
        # ds_exp = ds_exp.rename_vars({'lev_bounds': 'lev_bnds'})

    if 'lon_bounds' in cdnc.data_vars:
        cdnc = cdnc.rename_vars({'lon_bounds': 'lon_bnds'})
        cdnc = cdnc.rename_vars({'lat_bounds': 'lat_bnds'})

In [5]:
if cdnc.cf['Z'].formula == 'p = a*p0 + b*ps':
    cdnc = geocomp.interpolation.interp_hybrid_to_pressure(data  =cdnc['cdnc'], 
                                                        ps        =cdnc['ps'], 
                                                        hyam      =cdnc['a'], 
                                                        hybm      =cdnc['b'], 
                                                        p0        =cdnc.get('p0', 100000.0),
                                               new_levels=ta.plev)
elif cdnc.cf['Z'].formula in ['p = ap + b*ps','p(n,k,j,i) = ap(k) + b(k)*ps(n,j,i)']:
    cdnc = geocomp.interpolation.interp_hybrid_to_pressure(data  =cdnc['cdnc'], 
                                                        ps        =cdnc['ps'], 
                                                        hyam      =cdnc['a'], 
                                                        hybm      =cdnc['b'], 
                                                        p0        =cdnc.get('p0', 1),
                                               new_levels=ta.plev)

elif cdnc.cf['Z'].formula == 'z = a + b*orog':
    from workflow.scripts.utils import pressure_to_height
    from functools import partial
    orog = cdnc['orog'].copy()
    pfull = xr.open_dataset(snakemake.input.pfull[0]).mean(dim='time', keep_attrs=True)
    
    valid_pressures = pfull.where(pfull['pfull'] > p2)
    cdnc = cdnc.assign(cdnc=cdnc['cdnc'].where(np.isnan(valid_pressures['pfull']),cdnc['cdnc']))
    z = (cdnc['lev'] + cdnc['orog']*cdnc['b']).copy()
    dz = z.where(np.isnan(valid_pressures['pfull'])==False, np.nan)
    dz = (dz.max(dim='lev') - dz.isel(lev=0))
else:
    print('formula not reconized')

In [6]:
if snakemake.rule == 'column_integrate_cdnc_UKESM':
    cdnc = cdnc.sum(dim='lev')*1e-6*dz
    cdnc = cdnc['cdnc']
else:
    with xr.set_options(keep_attrs=True):
        cdnc=cdnc.sel(plev=slice(None,p2)).sum(dim='plev')

        tm = ta.sel(plev=slice(None,p2)).mean(dim='plev')
    
    if snakemake.wildcards.model == 'IPSL-CM6A-LR-INCA':
        cdnc= cdnc['cdnc']
    
    p1 = ta.plev[0]

    Rair=287.058
    g=9.81
    dz = Rair/g*tm['ta']*(np.log(p1/p2))

    cdnc=cdnc*dz*1e-6

In [7]:
cdnc.attrs['units']='m-2'
cdnc.attrs['standard_name'] = 'vertically_integrated_number_concentration_of_cloud_liquid_water_particles_in_air'
cdnc.attrs['long_name'] = 'Vertically Integrated Cloud Liquid Droplet Number Concentration'
cdnc.attrs['model_top'] = f'top of intergration {p2/100} hPa'

In [8]:
out_ds = cdnc.to_dataset(name=dvar)
if 'plev' in out_ds.coords:
    out_ds = out_ds.drop('plev')
elif 'lev' in out_ds.coords:
    out_ds = out_ds.drop('lev')
out_ds.attrs = attrs
out_ds.attrs['history'] = f'{out_ds.attrs["history"]},{snakemake.rule} vertical intragration'
out_ds.attrs['variable_id'] = dvar

In [ ]:
out_ds.to_netcdf(snakemake.output.outpath)